In [1]:
import cobra

import d3flux as d3f
from d3flux.core.flux_layouts import render_model

from jinja2 import Template

custom_css = \
"""
{% for item in items %}
text#{{ item }} {
    font-weight: 900;
}
{% endfor %}
text.cofactor {
    fill: #778899;
}
"""

css = Template(custom_css).render(items=['succ_c', 'ac_c', 'etoh_c', 'for_c', 'co2_c', 'lac__D_c'])

In [22]:
model = cobra.io.load_json_model('asuc_v1.json')

# model.add_reaction(ecoli.reactions.ALDD2x)
# d3f.update_cofactors(model, ['nadh_c'])


model.reactions.EX_glc_e.lower_bound = -1
model.reactions.EX_xyl_e.lower_bound = -6

import itertools

for obj in itertools.chain(model.reactions, model.metabolites):
    try:
        del obj.notes['map_info']['flux']
    except KeyError:
        pass

model.reactions.PPCK.notes['map_info']['group'] = 2
model.reactions.MDH.notes['map_info']['group'] = 2
model.reactions.FUM.notes['map_info']['group'] = 2

model.reactions.PFL.notes['map_info']['group'] = 'ko'
model.reactions.ACKr.notes['map_info']['group'] = 'ko'

# Just a single example, but many metabolite names could be better aligned
model.metabolites.get_by_id('f6p_c').notes['map_info']['align'] = 'center left'


from cobra.flux_analysis import pfba
pfba(model)

,fluxes,reduced_costs
ACALD,0.000000,2.0
ACALDt,0.000000,-2.0
ACKr,-4.333258,2.0
ACt2r,-4.333258,2.0
ALCD2x,0.000000,2.0
...,...,...
LDH_D,0.000000,-2.0
D_LACt2,0.000000,-2.0
EX_lac__D_e,0.000000,20.0
G6PDH2r,0.000000,2.0


In [23]:
html = d3f.flux_map(model, custom_css=css, figsize=(520, 660), default_flux_width=2.5, fontsize=14)

In [24]:
html